# DOMをtreeで実装してみる

- なお、パーサーは作らない(作るの結構大変らしい)
- [ブラウザのしくみ: データ構造とアルゴリズムと雰囲気で理解する DOM と Shadow DOM ](https://hayatoito.github.io/2017/dom/)を参考にする


## 見落としがちな点

- htmlタグはには内部に文字列を挟むものと、挟まないものの２種類がある。
    - ①`<div>●●通販の△△</div>`
    - ②`<button type="submit" value="送信">`
    
=> 
- `<div>●●通販の△△</div>`に関しては、`<div>`内部の`●●通販の△△`で1Node。`<div>●●通販の△△</div>`でそれをwrapしてまた一つのNode。

## Nodeは３種類ある(もっとあるかも)

|-|tag|value|children|
|----|----|----|----|
|①`<div>●●</div>`|div|None|[x,y,z]|
|②`<button type="submit">`|button|None|None|
|③文字列|None|その文字列|None|


## 実装方針

- Node<valueのみ>か、<tagとchildren>どちらかしか持たない？？？

# NodeとTreeの実装(簡易版)



- できたらtagはenumとかで持っておきたい(以下ではstringで持っている)

In [1]:
class Node:
    def __init__(self, tag, value, children):
        # このnodeが内部にもうnodeを持たない場合(つまり<>~</>を含まない文字列の場合)
        self.value = value
        self.tag = tag        
        # このnodeが内部にnode(s)を持つ場合
        self.children = children

class Tree:
    def __init__(self, node):
        self.root = node

## 実際に使ってみた

後のテストのために、<html>タグが二つあるが多めにみてほしい


```
<html>
  <html>
    <meta charset="utf-8" />
    あああああ
    <link rel="stylesheet" href="sample.css" type="text/css">
  </html>
  いいいいい
</html>
```

In [2]:
n1 = Node('meta', None, None)
n2 = Node(None, 'あああああ', None)
n3 = Node('link', None, None)

n4 = Node('html', None, [n1, n2, n3])
n5 = Node(None, 'いいいいい', None)

n6 = Node('html', None, [n4, n5])

tree = Tree(n6)

## count_tagsの実装

- treeのインスタンスメソッドとして作ってもいいが、今回はnodeのインスタンスメソッドとして作る
- あってるか自信ない

In [3]:
class Node:
    def __init__(self, tag, value, children):
        # このnodeが内部にもうnodeを持たない場合(つまり<>~</>を含まない文字列の場合)
        self.value = value
        self.tag = tag        
        # このnodeが内部にnode(s)を持つ場合
        self.children = children
        
    def count_tags(self, tag):
        num_tags = 1 if self.tag == tag else 0
        if self.tag is None:
            # 文字列なので、再帰を停止
            return num_tags
        # これも再帰の停止条件
        if self.children is None:
            return num_tags
        
        for each_node in self.children:
            num_tags += each_node.count_tags(tag)
        return num_tags

In [4]:
n1 = Node('meta', None, None)
n2 = Node(None, 'あああああ', None)
n3 = Node('link', None, None)

n4 = Node('html', None, [n1, n2, n3])
n5 = Node(None, 'いいいいい', None)

n6 = Node('html', None, [n4, n5])

n6.count_tags('html')

2

一応いけてる!...?

In [5]:
class Node:
    # parentとnext_nodeは基本的に初期化の時点ではわからないはずなので引数に含めない
    def __init__(self, tag, value):
        self.tag = tag
        self.value = value
        self.next_node = None
        self.previous_node = None
        self.first_child = None
        self.parent_node = None
        
    # 引数で与えられるタグが、selfのnodeとそれ以下のnodeたちにいくつ含まれるかを返す
    def count_tags(self, tag):
        num_tags = 1 if self.tag == tag else 0
        
        if self.first_child is not None:
            num_tags += self.first_child.count_tags_of_self_and_children_and_younger_brothers(tag)
        
        return num_tags        
    
    # そのnodeと、それよりも子供か弟のnodeに含まれる、引数で指定されたtagの個数を返す
    def count_tags_of_self_and_children_and_younger_brothers(self, tag):
        
        num_tags = 1 if self.tag == tag else 0
        
        # 再帰の停止条件、Nodeが自分よち年下の兄弟も、子供も持たない場合
        if (self.next_node is None) & (self.first_child is None):
            return num_tags
    
        if self.next_node is not None:
            num_tags += self.next_node.count_tags_of_self_and_children_and_younger_brothers(tag)
        
        if self.first_child is not None:
            num_tags += self.first_child.count_tags_of_self_and_children_and_younger_brothers(tag)
        
        return num_tags
        
        
# パーサーによって使用されることを推定した所謂helperメソッドである
# Nodeクラスのメソッドで、set_parent_and_siblings_relationship(self, child_nodes)を含めることも考えたが、
# インスタンスメソッド内で他のインスタンスのフィールドを書き換えるのはよくない気がしたので、helperメソッドで切り出す。
#
# 1. 第一引数のNodeを親Node, 第二引数の配列に含まれるNodeの配列をその子供達として、親子関係を持たせる。
# 2. 第二引数の配列に関しては、「配列で自分より前にあるNodeは自分より年上である」という兄弟関係を持たせる。
#    (要は1番目が一番年上、2番目が二番目に年上、、、)
# 
# Arguments:
#    parent_node(Node): 親となるNode
#    child_nodes(List of Node(s)): 子供たちとなるNodeのリスト(長さは最低1)
def set_parent_and_siblings_relationship(parent_node, child_nodes):
    
    previous_child = child_nodes[0]
    parent_node.first_child = previous_child
    previous_child.parent = parent_node
    for each_child in child_nodes[1:]:
        # 兄弟関係
        previous_child.next_node = each_child
        each_child.previous_node = previous_child
        # 親子関係
        each_child.parent = parent_node
        previous_child = each_child
    # 末っ子nodeについて
    parent_node.last_child = previous_child

    
class Tree:
    def __init__(self, node):
        self.root = node

## 実際に使ってみた
後のテストのために、タグが二つあるが多めにみてほしい

```
<html>(9)
  <html>(5)
    <meta charset="utf-8" />(1)
    あああああ(2)
    <link rel="stylesheet" href="sample.css" type="text/css">(3)
    <html>(4)
      いいいいい(0)
    </html>
  </html>
  ううううう(6)
  <html>(8)
    <meta charset="utf-8" />(7)
  </html>
</html>
```

In [6]:
# 引数は順にtag, value, first_child, last_child, previous_node

n0 = Node(None, 'いいいいい')

n1 = Node('meta', None)
set_parent_and_siblings_relationship(n1, [n0])
n2 = Node(None, 'あああああ')
n3 = Node('link', None)
n4 = Node('html', None)

n5 = Node('html', None)
set_parent_and_siblings_relationship(n5, [n1, n2, n3, n4])
n6 = Node(None, 'ううううう')
n7 = Node('meta', None)
n8 = Node('html', None)
set_parent_and_siblings_relationship(n8, [n7])

n9 = Node('html', None)
set_parent_and_siblings_relationship(n9, [n5, n6, n8])

tree = Tree(n9)

## count_tagsの実装

- treeのインスタンスメソッドとして作ってもいいが、今回はnodeのインスタンスメソッドとして作る
- あってるか自信ない

In [7]:
print(n9.count_tags('html'))
print(n8.count_tags('html'))
print(n0.count_tags('html'))

4
1
0


いけたっぽい!..?

## テストケースはどうする？

- 以下のテスト、そもそもまずset_parent_and_siblings_relationship関数をテストしないといけないな
    - 今回はやらないけど

- ほとんどの責務が`count_tags_of_self_and_children_and_younger_brothers()メソッド`でなされているので、そっちのテストを書いた方がよかったかもしれない
    - 今回はやらないけど

In [8]:
# nodeがchildrenもnextも持っていない場合
n0 = Node(None, 'あああああ')
assert n0.count_tags('test') == 0
n1 = Node('test', None)
assert n1.count_tags('test') == 1

# nodeが一つのみchildのみ持っている場合
n2 = Node('test', None)
n3 = Node('test', 'ううううう')
set_parent_and_siblings_relationship(n2, [n3])
assert n2.count_tags('test') == 2

# nodeが二つ以上のchildを持っている場合
n4 = Node('test', None)
n5 = Node(None, 'いいいいい')
n6 = Node('test', None)
set_parent_and_siblings_relationship(n4, [n5, n6])
assert n4.count_tags('test') == 2


# nodeが兄弟を持っていても、それがnodeのcountに含まれないことの確認
n_parent1 = Node('test', None)
n9 = Node('test', None)
n10 = Node(None, 'えええええ')
n11 = Node('test', None)
set_parent_and_siblings_relationship(n_parent1, [n9, n10, n11])
assert n9.count_tags('test') == 1